**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/pick.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200826'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  !pip install pyngrok
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)


# Inspecting the kinematic tree

Here is a simple example that demonstrates how to inspect the kinematic tree stored in a `MultibodyPlant`.

In [ ]:
from pydrake.all import FindResourceOrThrow, MultibodyPlant, Parser, plot_graphviz

plant = MultibodyPlant(time_step=0.0)
parser = Parser(plant)
parser.AddModelFromFile(FindResourceOrThrow(
    "drake/manipulation/models/allegro_hand_description/sdf/allegro_hand_description_right.sdf"))
parser.AddModelFromFile(FindResourceOrThrow(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf"))
plant.Finalize()

# TODO(russt): Update this to a javascript rendering.
# TODO(russt): Add floating base connections.
plot_graphviz(plant.GetTopologyGraphvizString());

# Forward kinematics of the iiwa + WSG

Here is a simple example that let's you visualize the frames on the iiwa and the gripper.  If you click on the "Open Controls" menu in the MeshCat visualizer, and dig into the menu `meshcat->drake->Source` then you will see elements for each of the models in the `SceneGraph`: one for the iiwa, another for the WSG, and others for the clutter bins.  You can enable/disable their visualization.  Give it a spin!

TODO(russt): Add some joint sliders here, too.

In [ ]:
from pydrake.all import DiagramBuilder, ConnectMeshcatVisualizer
from pydrake.examples.manipulation_station import ManipulationStation

builder = DiagramBuilder()

# TODO: Replace this with a simple model directive of iiwa+wsg (no clutter bins)
station = builder.AddSystem(ManipulationStation())
station.SetupClutterClearingStation()
station.Finalize()

frames_to_draw = {"iiwa": {"iiwa_link_1", "iiwa_link_2", "iiwa_link_3", "iiwa_link_4", "iiwa_link_5", "iiwa_link_6", "iiwa_link_7"},
                  "gripper": {"body"}}
meshcat = ConnectMeshcatVisualizer(builder,
    station.get_scene_graph(),
    output_port=station.GetOutputPort("pose_bundle"),
    zmq_url=zmq_url,
    frames_to_draw=frames_to_draw,
    axis_length=0.3,
    axis_radius=0.01)
meshcat.load()

diagram = builder.Build()
context = diagram.CreateDefaultContext()

diagram.Publish(context)